In [1]:
! pip install langchain_community
! pip install langchain_text_splitter
! pip install langchain_openai
! pip install faiss-cpu

  Using cached langchain_core-0.3.10-py3-none-any.whl.metadata (6.3 kB)
   ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
   ---------------------- ----------------- 1.3/2.4 MB 7.4 MB/s eta 0:00:01
   ---------------------------------------- 2.4/2.4 MB 7.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------------------------------------- 1.0/1.0 MB 6.8 MB/s eta 0:00:00
Using cached langchain_core-0.3.10-py3-none-any.whl (404 kB)
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.2.2
    Uninstalling langchain-text-splitters-0.2.2:
      Successfully uninstalled langchain-text-splitters-0.2.2


ERROR: Could not find a version that satisfies the requirement langchain_text_splitter (from versions: none)
ERROR: No matching distribution found for langchain_text_splitter


In [3]:
from dotenv import load_dotenv

# API KEY 정보 로드
load_dotenv()

import os

current_path = os.getcwd()
print("Current path:", current_path)

import bs4
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.document_loaders import PyMuPDFLoader

# 저장된 벡터스토어 로드
vectorstore = FAISS.load_local('./db/faiss', OpenAIEmbeddings(), allow_dangerous_deserialization=True)

# 로드된 벡터스토어에서 retriever 생성
retriever = vectorstore.as_retriever()

# 검색 예시
query = "토마토"
results = retriever.get_relevant_documents(query)

# 검색 결과 출력
for result in results:
    print(result.page_content)

from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    """당신은 질문-답변(Question-Answering)을 수행하는 친절한 AI 어시스턴트입니다. 당신의 임무는 주어진 문맥(context) 에서 주어진 질문(question) 에 답하는 것입니다.
검색된 다음 문맥(context) 을 사용하여 질문(question) 에 답하세요. 만약, 주어진 문맥(context) 에서 답을 찾을 수 없다면, 답을 모른다면 `주어진 정보에서 질문에 대한 정보를 찾을 수 없습니다` 라고 답하세요.
한글로 답변해 주세요. 단, 기술적인 용어나 이름은 번역하지 않고 그대로 사용해 주세요.

#Question:
{question}

#Context:
{context}

#Answer:"""
)

llm = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

# 체인을 생성합니다.
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

question = "30분 이내에 만들 수 있는 중식 요리 방법 알려줘"
answer = rag_chain.invoke(question)
print(answer)

question = "토마토를 이용한 간단한 요리 추천해줘"
answer = rag_chain.invoke(question)
print(answer)

question = "고추를 이용한 요리 추천해줘"
answer = rag_chain.invoke(question)
print(answer)


Current path: c:\Users\carpe\OneDrive\Desktop\p\rag


C:\Users\carpe\AppData\Local\Temp\ipykernel_16660\1397806946.py:29: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  results = retriever.get_relevant_documents(query)


title: 아침에 한잔 마시면 든든하지요! 토마토주스
servings: 3 servings
cooking_time: PT10M
ingredients: 토마토 4개,꿀 2T,얼음 2Cup
cooking_orders: ['1. 토마토는 4등분하여 꼭지만 잘라 준비한다.', '2. 믹서기에 토마토, 야쿠르트, 꿀, 얼음을 넣어 갈아준다.', '3. 모두 갈리면 컵에 넣어 완성한다.']
url: https://www.10000recipe.com//recipe/6871686
﻿title: 토마토가 방울방울해 ♡♥♡♥ 토마토매실절임 ★
servings: 6 servings
cooking_time: PT20M
ingredients: 방울 토마토 1팩,레몬 1/2개,매실청 250ml,바질 약간 (또는 로즈마리)
cooking_orders: ['1. 방울토마토는 꼭지를 따고 밑둥에 십자모양으로 칼집을 내요.', '2. 레몬은 슬라이스로 썰어요.', '3. 냄비에 물을 넣어 끓어오르면 방울토마토를 넣어요.', '4. 십자모양의 껍질이 벗겨지기 시작하면 건져 찬물에 넣어요.', '5. 방울토마토의 껍질을 벗겨 키친타월로 물기를 제거해요.', '6. 열탕소독한 유리병에 방울토마토, 레몬, 바질 또는 로즈마리를 넣어요.', '7. 매실청을 부어 뚜껑을 닫아 완성해요.']
url: https://www.10000recipe.com//recipe/6917240
﻿title: 토마토스튜 같은 맛! 묘한 매력이 있는 토마토카레 ♥
servings: 2 servings
cooking_time: PT30M
ingredients: 소고기 150g,토마토 1개,감자 1개,양파 1개,물 2종이컵,토마토소스 1종이컵,고형카레 3조각,맛술 1/2숟가락,소금 약간,후추 약간,플레인요거트 2숟가락
cooking_orders: ['1. 껍질을 벗긴 감자, 양파는 깍둑 썰고 토마토도 한 입 크기로 썬다.', '2. 핏물을 제거한 쇠고기는 밑간 재료를 넣고 밑간한다.', '3. 달군 냄비에 올리브유를 두르고 